In [31]:
import pandas as pd
import numpy as np
import requests
import re
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
import time

In [34]:
URL = 'https://novosibirsk.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=4897&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
link_list = []

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36')
options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(0.05)

for i in range(1, 11):
    time.sleep(0.05)
    html = driver.find_element(By.TAG_NAME, "html")
    time.sleep(0.05)

    for k in range(400):
        html.send_keys(Keys.DOWN)

    offers = driver.find_elements(By.CLASS_NAME, '_93444fe79c--link--VtWj6')

    for elem in offers:
        link_list.append(elem.get_attribute('href'))
    
    next_page = f'https://novosibirsk.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={i+1}&region=4897&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
    driver.get(next_page)

In [47]:
flat_list = []
driver = webdriver.Chrome(options=options)

for link in link_list[:10]:
    driver.get(link)
    flat = {}
    address_str = ''

    descript = driver.find_element(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[2]/div[3]')
    elements = descript.find_elements(By.CLASS_NAME, 'a10a3f92e9--item--Jp5Qv')
    price = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--amount--ON6i1')
    address = driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--address--SMU25')

    try:
        metro = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--undergrounds--sGE99')
    except:
        pass

    for elem in address:
        address_str += elem.text

    for item in elements:
        item = item.text.split('\n')
        flat[item[0]] = item[1]

    flat['price'] = price.text
    flat['metro'] = metro.text
    flat['address'] = address_str
    flat_list.append(flat)